In [1]:
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf

Con esta línea bajo y leo los datos de MNIST automáticamente

In [15]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)


Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


Los datos están separados en 3 partes

- `mnist.train`: 55k de datos de **entrenamiento**
- `mnist.test`: 10k de datos de **prueba**
- `mnist.validation`: 5k de datos de **validación**

Cada dato de MNIST tiene dos partes: una imágen de un dígito dibujado y su etiqueta correspondiente. Las **imágenes** (* es una matriz de 55kx784*) son las **x** y las **etiquetas** (*una matriz de 55kx10*) son las **y**.

### Softmax Regressions

Para cada imágen de MNIST quiero tener la probabilidad de que sea cada dígito. Softmax me da una lista de valores entre 0 y 1 que suman 1.

Una regresión Softmaxtiene dos pasos

1. Juntar la evidencia de que el input esté en cierta clase.
    - Se hace una suma pesada de la intensidad de cada pixel. El peso es positivo si la evidencia es a favor y negativo si es en contra.
    - Se le agrega un bias para tener en cuenta los aspectos que son independientes del input.
2. Convertir la evidencia en probabilidades.
    - Esto se hace mediante la función softmax(x)=exp(xi)/Sum(exp(xj))


In [3]:
#x será un placeholder, es decir que el valor se lo doy como input cuando corra 
# el TF.
x = tf.placeholder(tf.float32, [None, 784])

## En general queremos que los parámetros del modelo sean `Variable`

W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))


Implementación de softmax

In [5]:
y = tf.nn.softmax(tf.matmul(x, W) + b)


### Entrenamiento

Para entrenar el modelo necesito encontrar la función de costo que tengo que minimizar.


Una función de costo es la **entropía cruzada** :

*H_y'(y)=-Sum(y'i Log(yi))*

Donde y se la distribución de probabilidad predicha y y' es la verdadera distribución.

La entropía cruzada es una medida de que tan ineficiente son nuestras predicciones para describir la verdad.

#### Implementación:

In [6]:
y_ = tf.placeholder(tf.float32, [None, 10]) #Acá voy a poner las respuestas correctas


In [10]:
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))

#-tf.log: es el logaritmo de cada y
#- después multiplico cada elemento de y_ con el logaritmo de y
#-tf.reduce_sum: agrega los elementos en la segunda dimensión de y ya que
#reduction_indices=[1]
#-tf.reduce_mean: calcula la media sobre cada uno de los ejemplos.


Trato de minimizar el costo. Para eso uso **Gradient Descent**

In [11]:
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)


Se **inicia el modelo**

In [12]:
sess = tf.InteractiveSession()


Tengo que inicializar las variables que creé

In [13]:
tf.global_variables_initializer().run()


Pongo a entrenar: corro los pasos de entrenamiento mil veces.

In [17]:
for _ in range(1000):
  batch_xs, batch_ys = mnist.train.next_batch(100)
  sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})


### Evaluación del modelo

Voy a usar la función tf.argmax porque me da el índice de la entrada 
más grande de un tensor en cierto eje.

In [20]:
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
#Esto me da una lista de booleanos

In [21]:
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))


Acá pregunto la precisión del modelo que tengo

In [22]:
print(sess.run(accuracy, feed_dict={x: nist.test.images, y_: nist.test.labels}))


0.9193
